# Pre-processing News Data

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [45]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [46]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [47]:
pip install pyvi

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyvi in c:\users\admin\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (0.1.1)




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [48]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [49]:
INPUT = r'C:\Users\ADMIN\Downloads\ML-homeworks\ML-homeworks\Hw1\news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

## hàm os.makedirs:
- là hàm trong thư viện os, được sử dụng để tạo thư mục (directory) mới.
    - `parent/child/grandchild`
- `os.listdir` là một hàm trong module os của Python, được sử dụng để liệt kê tất cả các tập tin và thư mục con trong một thư mục nhất định.


In [50]:
# Print header
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')

n = 0
for label in os.listdir(INPUT):
    label_path = os.path.join(INPUT, label)
    # Check if the item is a directory
    if os.path.isdir(label_path):
        num_files = len(os.listdir(label_path))
        print(f'{label}: {num_files}')
        n += num_files

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339


- `data_train`: tập dữ liệu để train

- `data_train.target`: (thường là số nguyên) dạng số dùng để huấn luyện và tính toán.

- `data_train.target_names`: Tên nhãn mục tiêu là dạng chuỗi văn bản dùng để giúp con người hiểu và diễn giải kết quả mô hình một cách dễ dàng.

- `data_train.data`: các đặc trưng (input features)

- `filenames`: cho biết nguồn gốc dữ liệu

In [51]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['C:\\Users\\ADMIN\\Downloads\\ML-homeworks\\ML-homeworks\\Hw1\\news_vnexpress\\khoa-hoc\\00133.txt']
[4]
['Mời độc giả đặt câu hỏi tại đây\n']

Tổng số  văn bản: 1339


## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

Using CountVectorizer to Extracting Features from Text:

vd: corpus D = [`one`, `two, one`, `three, two`]

turn into matrix TF:

word one two three

d1    1   0    0

d2    1   0    0

d3    0   1    1

In [52]:
# load dữ liệu các stopwords 

#---> Code ở đây
stop_words = ['a_lô', 'a_ha','ai', 'ai_ai', 'ai_nấy', 'ai_đó','alô', 'amen', 'anh','anh_ấy']
count = 0
for data in data_train.data:
    data = data.split(" ")
    for i in data:
        if i in stop_words:
            count += 1
            
print('Số lượng stopwords: '+ str(count) )

Số lượng stopwords: 1287


In [53]:
#---> Code ở đây
data_train.data = [word for word in data_train.data if word.lower() not in stop_words]
# Create a vectorize object:
vectorizer = CountVectorizer()
# vectorize.fit -> tạo 1 từ điển
vector_dict = vectorizer.fit(data_train.data)
# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
#vectorize.transform -> chuyển từ điển thành matrix
vector = vectorizer.transform(data_train.data)
print(vector_dict)
print(vector)

CountVectorizer()
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 321807 stored elements and shape (1339, 13110)>
  Coords	Values
  (0, 3137)	1
  (0, 4461)	1
  (0, 5286)	1
  (0, 7598)	1
  (0, 11581)	1
  (0, 12850)	1
  (0, 12973)	1
  (0, 13021)	1
  (1, 3210)	1
  (1, 4781)	1
  (1, 7727)	1
  (1, 7815)	1
  (1, 11260)	1
  (1, 11515)	1
  (1, 11540)	1
  (1, 12054)	1
  (1, 13068)	1
  (2, 1)	3
  (2, 81)	1
  (2, 82)	1
  (2, 168)	2
  (2, 392)	2
  (2, 556)	1
  (2, 600)	1
  (2, 769)	1
  :	:
  (1338, 12735)	1
  (1338, 12738)	2
  (1338, 12801)	7
  (1338, 12815)	1
  (1338, 12830)	1
  (1338, 12850)	1
  (1338, 12854)	2
  (1338, 12879)	1
  (1338, 12885)	1
  (1338, 12892)	1
  (1338, 12910)	1
  (1338, 12917)	2
  (1338, 12921)	2
  (1338, 12929)	1
  (1338, 12934)	2
  (1338, 12948)	6
  (1338, 12952)	1
  (1338, 12979)	4
  (1338, 12985)	1
  (1338, 12997)	1
  (1338, 13011)	5
  (1338, 13024)	1
  (1338, 13034)	1
  (1338, 13043)	2
  (1338, 13065)	1


In [54]:
# load dữ liệu các stopwords 

#---> Code ở đây
stop_words = ['a_lô', 'a_ha','ai', 'ai_ai', 'ai_nấy', 'ai_đó','alô', 'amen', 'anh','anh_ấy']
count = 0
for data in data_train.data:
    data = data.split(" ")
    for i in data:
        if i in stop_words:
            count += 1
            
print('Số lượng stopwords: '+ str(count) )
print(stop_words)
# Chuyển hoá dữ liệu text về dạng vector TF 
#     - loại bỏ từ dừng
#     - sinh từ điển

#---> Code ở đây
data_train.data = [word for word in data_train.data if word.lower() not in stop_words]
# Create a vectorize object:
count_vectorizer = CountVectorizer()
# vectorize.fit -> tạo 1 từ điển
count_vectorizer.fit(data_train.data)
# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
#vectorize.transform -> chuyển từ điển thành matrix
vector = count_vectorizer.transform(data_train.data)

# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 
tfidf_transformer = TfidfTransformer()  
tfidf_matrix = tfidf_transformer.fit_transform(vector)
data_preprocessed = tfidf_matrix #Code ở đây

X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(count_vectorizer.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")

Số lượng stopwords: 1287
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 13110
Kích thước dữ liệu sau khi xử lý: (1339, 13110)
Kích thước nhãn tương ứng: (1339,)


In [55]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.11842433 0.        ]]
5


In [56]:
sum(sum(X[100].toarray() != 0))

np.int64(429)

In [57]:
print(X[100])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 429 stored elements and shape (1, 13110)>
  Coords	Values
  (0, 81)	0.012822585650213027
  (0, 100)	0.01750765078081801
  (0, 156)	0.019609395414955842
  (0, 188)	0.039808213738317146
  (0, 269)	0.028019904420274182
  (0, 392)	0.020891492057197123
  (0, 397)	0.028856603910450362
  (0, 418)	0.041235391541495005
  (0, 662)	0.01919386883204624
  (0, 909)	0.027970913654820045
  (0, 1195)	0.04313391756443159
  (0, 1210)	0.08626783512886319
  (0, 1220)	0.04313391756443159
  (0, 1272)	0.013491978833492185
  (0, 1320)	0.011096668088260256
  (0, 1531)	0.012521999334879425
  (0, 1593)	0.029441331479509596
  (0, 1634)	0.019804180634220174
  (0, 1786)	0.03990945868727483
  (0, 1791)	0.03915405678574102
  (0, 1870)	0.04313391756443159
  (0, 2050)	0.03443252290167865
  (0, 2062)	0.06057955554406974
  (0, 2087)	0.012262959831398938
  (0, 2113)	0.022706261085952147
  :	:
  (0, 12858)	0.027026891794766736
  (0, 12866)	0.028019904420274182
  